In [40]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from pydantic import Field
from langgraph.graph.message import add_messages   # Special reducer function built into langgraph
from langgraph.checkpoint.memory import MemorySaver  # RAM Memory to save conversations

In [13]:
# Define State
class ChatState(TypedDict):

  messages: Annotated[List[BaseMessage], add_messages]    # Reducer function

In [15]:
from google.colab import userdata
api_key = userdata.get('OPEN_AI')

In [17]:
# Defining nodes
llm = ChatOpenAI(api_key=api_key)

def chat_node(state : ChatState):
  # Take user query from state
  messages = state['messages']
  # Send to LLM
  response = llm.invoke(messages)
  # Response store in state
  return {'messages':[response]}

In [42]:
checkpointer = MemorySaver()

graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

In [44]:
# Chatbot interaction loop
thread_id = '1' # Unique identifier for the conversation thread

# Start an infinite loop for continuous interaction
while True:
  # Prompt the user for input
  user_message = input('Type here: ')

  # Check if the user wants to exit the conversation
  if user_message.strip().lower() in ['bye', 'exit']:
    break

  # Configure the chatbot with the current thread ID for memory
  config = {'configurable': {'thread_id': thread_id}}


  response = chatbot.invoke({'messages': HumanMessage(content = user_message)}, config=config)

  print(response['messages'][-1].content)

Type here: What is the capital of France?
The capital of France is Paris.
Type here: Tell me something special about it
Paris is often called the "City of Light" because it was one of the first cities in the world to have street lighting on a grand scale. The nickname also refers to Paris' role as a center of education and ideas during the Age of Enlightenment in the 18th century. Today, Paris is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and Louvre Museum, as well as its rich artistic and cultural history.
Type here: exit


In [37]:
# Chatbot -  Simple intuition without memory
initial_state = {
    'messages': []
}


while True:
  user_message = input("Type Here: ")

  if user_message.strip().lower() in ['exit', 'bye', 'quit']:
    break

  initial_state['messages'].append(HumanMessage(content = user_message))
  initial_state = chatbot.invoke(initial_state)
  print(initial_state['messages'][-1].content)

Type Here: Hi, my name is Feroz
Hello Feroz, nice to meet you! How can I assist you today?
Type Here: Can you tell me my name?
Yes, your name is Feroz.
Type Here: I  love to eat icecream
That's great, ice cream is a delicious treat! What is your favorite flavor of ice cream?
Type Here: I am 27 years old
That's nice to know. Is there anything else you would like to share or ask about?
Type Here: what do you knwo about me?
I only know the information that you have shared with me, such as your name being Feroz and that you are 27 years old. I do not have access to any personal information about you other than what you provide during our conversation. Is there anything specific you would like to know or discuss?
Type Here: add 10 to 10
10 + 10 equals 20.
Type Here: add 15
15 to what?
Type Here: the earlier result
Adding 15 to the earlier result of 20 gives us a total of 35.
Type Here: bye


In [39]:
# Persistance in LangGraph - Memory
